In [1]:
!pip install pyunpack patool pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 1.9 MB/s eta 0:00:00


In [2]:
!wget -O dataset.rar https://github.com/NLP-Final-Projects/SSPP/raw/main/Phase%201%20-%20Data%20Collection/nlp-finalproject-sspp-dataset.rar

--2024-07-30 05:52:40--  https://github.com/NLP-Final-Projects/SSPP/raw/main/Phase%201%20-%20Data%20Collection/nlp-finalproject-sspp-dataset.rar
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/NLP-Final-Projects/SSPP/main/Phase%201%20-%20Data%20Collection/nlp-finalproject-sspp-dataset.rar [following]
--2024-07-30 05:52:41--  https://raw.githubusercontent.com/NLP-Final-Projects/SSPP/main/Phase%201%20-%20Data%20Collection/nlp-finalproject-sspp-dataset.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102179768 (97M) [application/octet-stream]
Saving to: ‘dataset.rar’

dataset.rar         100%[==

In [3]:
from pyunpack import Archive
import os

extract_dir = 'extracted_files'
os.makedirs(extract_dir, exist_ok=True)

Archive('dataset.rar').extractall(extract_dir)

In [4]:
import pandas as pd
import glob

csv_files = glob.glob(os.path.join(extract_dir, '**', '*.csv'), recursive=True)

dfs = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

In [5]:
import shutil

shutil.rmtree(extract_dir)

In [6]:
os.remove('dataset.rar')

In [7]:
filtered_df = merged_df.dropna(subset=["Functionality", "Publications"], how='all')

sorted_df = filtered_df.sort_values(by="PDB ID")

df = sorted_df

In [8]:
print(len(df))

156588


In [9]:
df.head()

,PDB ID,Protein Name,UniProt ID,Functionality,Publications,Secondary Structures Q8
89916,101M,Myoglobin,P02185,Monomeric heme protein which primary function ...,NaN,"[('M', '-'), ('V', '-'), ('L', '-'), ('S', '-'..."
89917,102L,Endolysin,P00720,Endolysin with lysozyme activity that degrades...,NaN,"[('M', '-'), ('N', '-'), ('I', 'H'), ('F', 'H'..."
89918,102M,Myoglobin,P02185,Monomeric heme protein which primary function ...,NaN,"[('M', '-'), ('V', '-'), ('L', '-'), ('S', '-'..."
89919,103L,Endolysin,P00720,Endolysin with lysozyme activity that degrades...,NaN,"[('M', '-'), ('N', '-'), ('I', 'H'), ('F', 'H'..."
89920,103M,Myoglobin,P02185,Monomeric heme protein which primary function ...,NaN,"[('M', '-'), ('V', '-'), ('L', '-'), ('S', '-'..."


In [10]:
df = df.drop_duplicates(subset=["UniProt ID", "Secondary Structures Q8"])

In [11]:
print(len(df))

115495


In [12]:
!pip install nltk

In [13]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [14]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [15]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

In [16]:
def preprocess_column(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: preprocess_text(x) if pd.notna(x) else x)
    return df

In [17]:
df = preprocess_column(df, "Functionality")
df = preprocess_column(df, "Publications")

<ipython-input-16-343d15b3f94c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(lambda x: preprocess_text(x) if pd.notna(x) else x)


In [19]:
print(df[['Functionality', 'Publications']][90000:91000])

                                            Functionality Publications
72342   attache virion cell membrane interacting host ...          NaN
72343   attache virion cell membrane interacting host ...          NaN
18593   phosphorylates protein serine threonine tyrosi...          NaN
18594   rna polymerase rnap catalyzes transcription dn...          NaN
124094  potent cytokine initially discovered major end...          NaN
...                                                   ...          ...
116769  protein mediates sodium ion permeability excit...          NaN
52192   modulates channel gating kinetics cause negati...          NaN
13303   lamins intermediate filament protein assemble ...          NaN
52195   rna polymerase catalyzes transcription dna rna...          NaN
52196   rna polymerase catalyzes transcription dna rna...          NaN

[1000 rows x 2 columns]


In [27]:
df['Functionality'] = df.apply(
    lambda row: f"{row['Functionality']} {row['Publications']}" if pd.notna(row['Publications']) else row['Functionality'],
    axis=1
)

df.drop(columns=['Publications'], inplace=True)

<ipython-input-27-3521413d37d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Functionality'] = df.apply(
<ipython-input-27-3521413d37d0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Publications'], inplace=True)


In [20]:
#!pip install ydata-profiling

In [21]:
# import ydata_profiling

# profile = ydata_profiling.ProfileReport(df)

# profile

In [29]:
df.head(10)

,PDB ID,Protein Name,UniProt ID,Functionality,Secondary Structures Q8
89916,101M,Myoglobin,P02185,monomeric heme protein primary function store ...,"[('M', '-'), ('V', '-'), ('L', '-'), ('S', '-'..."
89917,102L,Endolysin,P00720,endolysin lysozyme activity degrades host pept...,"[('M', '-'), ('N', '-'), ('I', 'H'), ('F', 'H'..."
89918,102M,Myoglobin,P02185,monomeric heme protein primary function store ...,"[('M', '-'), ('V', '-'), ('L', '-'), ('S', '-'..."
89919,103L,Endolysin,P00720,endolysin lysozyme activity degrades host pept...,"[('M', '-'), ('N', '-'), ('I', 'H'), ('F', 'H'..."
89920,103M,Myoglobin,P02185,monomeric heme protein primary function store ...,"[('M', '-'), ('V', '-'), ('L', '-'), ('S', '-'..."
89921,104L,Endolysin,P00720,endolysin lysozyme activity degrades host pept...,"[('M', '-'), ('N', '-'), ('I', 'H'), ('F', 'H'..."
89922,104M,Myoglobin,P02185,monomeric heme protein primary function store ...,"[('V', '-'), ('L', '-'), ('S', '-'), ('E', 'H'..."
89923,105M,Myoglobin,P02185,monomeric heme protein primary function store ...,"[('V', '-'), ('L', '-'), ('S', '-'), ('E', 'H'..."
89924,106M,Myoglobin,P02185,monomeric heme protein primary function store ...,"[('M', '-'), ('V', '-'), ('L', '-'), ('S', '-'..."
89925,107L,Endolysin,P00720,endolysin lysozyme activity degrades host pept...,"[('M', '-'), ('N', '-'), ('I', 'H'), ('F', 'H'..."


In [28]:
df.to_csv('dataset.csv', index=False)

In [24]:
!apt-get install rar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  rar
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 341 kB of archives.
After this operation, 1,202 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 rar amd64 2:5.5.0-1.1 [341 kB]
Fetched 341 kB in 1s (357 kB/s)
Selecting previously unselected package rar.
(Reading database ... 123589 files and directories currently installed.)
Preparing to unpack .../rar_2%3a5.5.0-1.1_amd64.deb ...
Unpacking rar (2:5.5.0-1.1) ...
Setting up rar (2:5.5.0-1.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [30]:
!rar a dataset.rar /content/dataset.csv


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive dataset.rar

Adding    /content/dataset.csv                                             0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 80% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92%